In [48]:
import os
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np

In [3]:
# Parsing the XML file
directory_path = './'

xml_path = os.path.join(directory_path, 'export.xml')
tree = ET.parse(xml_path)
root = tree.getroot()

In [6]:
record_list = [x.attrib for x in root.iter('Record')]
df_raw = pd.DataFrame(record_list)
print(df_raw.columns.tolist())

['type', 'sourceName', 'sourceVersion', 'unit', 'creationDate', 'startDate', 'endDate', 'value', 'device']


In [12]:
cols_to_avoid = [
    'creationDate',
    'startDate',
    'endDate',
    'value'
]

# Abrindo um arquivo de texto para escrita
with open('unique_values.txt', 'w') as file:
    for col in df_raw.columns:
        if col not in cols_to_avoid:
            # Escrevendo no arquivo ao invés de imprimir no console
            file.write(col + '\n')
            unique_values = list(df_raw[col].unique())
            file.write(str(unique_values) + '\n\n')

print("A saída foi salva no arquivo 'unique_values.txt'")


A saída foi salva no arquivo 'unique_values.txt'


In [46]:
query_df = df_raw.query('type == "HKQuantityTypeIdentifierHeartRateVariabilitySDNN"').sort_values(by='startDate')
print(len(query_df))
print(query_df.sourceName.unique())s
print(query_df.value.unique()[:10])
query_df.tail()


6147
['Apple\xa0Watch de Breno']
['35.5114' '73.9493' '77.3034' '65.2294' '72.9332' '123.201' '39.9547'
 '76.3291' '55.0466' '40.0515']


,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device
1967808,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Apple Watch de Breno,10.2,ms,2024-01-04 07:34:22 -0300,2024-01-04 07:33:21 -0300,2024-01-04 07:34:21 -0300,48.5585,"<<HKDevice: 0x28146fb10>, name:Apple Watch, ma..."
1967809,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Apple Watch de Breno,10.2,ms,2024-01-04 11:52:35 -0300,2024-01-04 11:51:33 -0300,2024-01-04 11:52:33 -0300,27.136,"<<HKDevice: 0x28146fb10>, name:Apple Watch, ma..."
1967810,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Apple Watch de Breno,10.2,ms,2024-01-04 13:44:18 -0300,2024-01-04 13:43:16 -0300,2024-01-04 13:44:15 -0300,32.0706,"<<HKDevice: 0x28146fb10>, name:Apple Watch, ma..."
1967811,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Apple Watch de Breno,10.2,ms,2024-01-04 15:33:47 -0300,2024-01-04 15:32:46 -0300,2024-01-04 15:33:45 -0300,91.8269,"<<HKDevice: 0x28146fb10>, name:Apple Watch, ma..."
1967812,HKQuantityTypeIdentifierHeartRateVariabilitySDNN,Apple Watch de Breno,10.2,ms,2024-01-04 17:34:07 -0300,2024-01-04 17:33:06 -0300,2024-01-04 17:34:04 -0300,59.2398,"<<HKDevice: 0x28146fb10>, name:Apple Watch, ma..."


In [53]:
ignored_types = [x for x in df_raw['type'].unique() if x not in granular_cols + group_day_average_cols + group_day_sum_cols + group_day_value_sum]

df = df_raw.loc[~df_raw['type'].isin(ignored_types)]
print(len(df_raw))
print('len df: ', len(df))
print(df.head())

1967813
len df:  1786368
                               type sourceName sourceVersion unit  \
2  HKQuantityTypeIdentifierBodyMass     Health        16.1.1   kg   
3  HKQuantityTypeIdentifierBodyMass     Health        16.1.1   kg   
4  HKQuantityTypeIdentifierBodyMass     Health        16.1.1   kg   
5  HKQuantityTypeIdentifierBodyMass     Health        16.1.1   kg   
6  HKQuantityTypeIdentifierBodyMass     Health        16.1.1   kg   

                creationDate                  startDate  \
2  2023-01-18 07:42:03 -0300  2023-01-18 07:41:00 -0300   
3  2023-01-19 08:03:52 -0300  2023-01-19 08:03:00 -0300   
4  2023-01-20 08:38:13 -0300  2023-01-20 08:38:00 -0300   
5  2023-01-22 09:55:18 -0300  2023-01-22 09:55:00 -0300   
6  2023-01-23 08:01:00 -0300  2023-01-23 08:00:00 -0300   

                     endDate value device  
2  2023-01-18 07:41:00 -0300  81.9    NaN  
3  2023-01-19 08:03:00 -0300  81.2    NaN  
4  2023-01-20 08:38:00 -0300  80.8    NaN  
5  2023-01-22 09:55:00 -0300 

In [ ]:
# Ensure that startDate and endDate are datetime objects
df['startDate'] = pd.to_datetime(df['startDate'])
df['endDate'] = pd.to_datetime(df['endDate'])

In [22]:
df['duration_minutes'] = (df['endDate'] - df['startDate']).dt.seconds / 60
df['duration_hours'] = df['duration_minutes'] / 60

In [35]:
df.loc[~df['sourceName'].isin(['Zepp', 'Sleep Cycle'])].to_pickle('df.pickle')

In [78]:
df: pd.DataFrame = pd.read_pickle('df.pickle')

In [79]:
df.dtypes

type                                   object
sourceName                             object
sourceVersion                          object
unit                                   object
creationDate                           object
startDate           datetime64[ns, UTC-03:00]
endDate             datetime64[ns, UTC-03:00]
value                                  object
device                                 object
duration_minutes                      float64
duration_hours                        float64
dtype: object

In [80]:
MAX_HOURS_ALLOWED = 10.5

df = df.loc[df['duration_hours'] < MAX_HOURS_ALLOWED]

numeric_values = pd.to_numeric(df['value'], errors='coerce')
df['value_fix'] = np.where(numeric_values.isna(), df['duration_minutes'], df['value'])
df['value_type'] = np.where(numeric_values.isna(), 'minutes', 'number')

In [81]:
# Convertendo 'value' para numérico, NaN onde não for possível
numeric_values = pd.to_numeric(df['value'], errors='coerce')

# Definindo condições
conditions = [
    numeric_values.notna(),  # Valores numéricos
    numeric_values.isna() & (df['duration_minutes'] == 0),  # Valores de categoria
    numeric_values.isna()  # Valores de duração
]

# Definindo escolhas para 'value_fix'
choices_value_fix = [
    df['value'],  # Manter o valor numérico original
    1,  # Substituir por um valor representativo da categoria ou pela própria categoria
    df['duration_minutes']  # Usar valor de duração
]

# Definindo escolhas para 'value_type'
choices_value_type = [
    'number',    # Tipo numérico
    'category',  # Tipo categoria
    'minutes'    # Tipo duração
]

# Aplicando as condições
df['value_fix'] = pd.to_numeric(np.select(conditions, choices_value_fix, default=np.nan))
df['value_type'] = np.select(conditions, choices_value_type, default='unknown')
df['start_date'] = df['startDate'].dt.date

In [82]:
granular_cols = [
    "HKQuantityTypeIdentifierBodyMass",
    "HKQuantityTypeIdentifierHeartRate",
    "HKQuantityTypeIdentifierRespiratoryRate",
    "HKQuantityTypeIdentifierBodyTemperature",
    "HKQuantityTypeIdentifierRestingHeartRate",
    "HKQuantityTypeIdentifierVO2Max",
    "HKQuantityTypeIdentifierSixMinuteWalkTestDistance",
    "HKCategoryTypeIdentifierMindfulSession",
    "HKCategoryTypeIdentifierHeadache",
    "HKCategoryTypeIdentifierPelvicPain",
    "HKCategoryTypeIdentifierSoreThroat",
    "HKCategoryTypeIdentifierSinusCongestion",
]

group_day_sum_cols = [
    "HKQuantityTypeIdentifierStepCount",
    "HKQuantityTypeIdentifierBasalEnergyBurned",
    "HKQuantityTypeIdentifierActiveEnergyBurned",
    "HKQuantityTypeIdentifierAppleExerciseTime",
    "HKQuantityTypeIdentifierDistanceCycling",
    "HKQuantityTypeIdentifierPhysicalEffort",
]

group_day_average_cols = [
    "HKQuantityTypeIdentifierWalkingStepLength",
    "HKQuantityTypeIdentifierHeartRateVariabilitySDNN"
]

group_day_value_sum = [
    "HKCategoryTypeIdentifierSleepAnalysis",
]



In [84]:
groupby_cols = ['type', 'sourceName', 'unit', 'start_date', 'value_type']

df_group_sum = df.loc[df['type'].isin(group_day_sum_cols)].groupby(groupby_cols)['value_fix'].sum().reset_index()
df_group_avg = df.loc[df['type'].isin(group_day_average_cols)].groupby(groupby_cols)['value_fix'].mean().reset_index()

In [105]:
df.loc[df['type'].isin(group_day_value_sum)]

,type,sourceName,sourceVersion,unit,creationDate,startDate,endDate,value,device,duration_minutes,duration_hours,value_fix,value_type,start_date
1930356,HKCategoryTypeIdentifierSleepAnalysis,Clock,NaN,NaN,2020-01-24 11:00:34 -0300,2020-01-24 07:38:06-03:00,2020-01-24 11:00:29-03:00,HKCategoryValueSleepAnalysisInBed,"<<HKDevice: 0x2817d8190>, name:iPhone, manufac...",202.383333,3.373056,202.383333,minutes,2020-01-24
1930357,HKCategoryTypeIdentifierSleepAnalysis,Clock,NaN,NaN,2020-01-24 11:00:39 -0300,2020-01-24 07:38:06-03:00,2020-01-24 11:00:34-03:00,HKCategoryValueSleepAnalysisInBed,"<<HKDevice: 0x2817d8190>, name:iPhone, manufac...",202.466667,3.374444,202.466667,minutes,2020-01-24
1930358,HKCategoryTypeIdentifierSleepAnalysis,Clock,NaN,NaN,2020-01-25 11:00:09 -0300,2020-01-25 03:51:02-03:00,2020-01-25 11:00:08-03:00,HKCategoryValueSleepAnalysisInBed,"<<HKDevice: 0x2817d8190>, name:iPhone, manufac...",429.100000,7.151667,429.100000,minutes,2020-01-25
1930359,HKCategoryTypeIdentifierSleepAnalysis,Clock,NaN,NaN,2020-01-26 11:09:46 -0300,2020-01-26 09:44:14-03:00,2020-01-26 11:02:32-03:00,HKCategoryValueSleepAnalysisInBed,"<<HKDevice: 0x2817d8190>, name:iPhone, manufac...",78.300000,1.305000,78.300000,minutes,2020-01-26
1930360,HKCategoryTypeIdentifierSleepAnalysis,Clock,NaN,NaN,2020-01-26 11:09:52 -0300,2020-01-26 09:44:14-03:00,2020-01-26 11:02:32-03:00,HKCategoryValueSleepAnalysisInBed,"<<HKDevice: 0x2817d8190>, name:iPhone, manufac...",78.300000,1.305000,78.300000,minutes,2020-01-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1947999,HKCategoryTypeIdentifierSleepAnalysis,Apple Watch de Breno,10.2,NaN,2024-01-04 08:12:23 -0300,2024-01-04 06:53:31-03:00,2024-01-04 07:55:31-03:00,HKCategoryValueSleepAnalysisInBed,NaN,62.000000,1.033333,62.000000,minutes,2024-01-04
1948000,HKCategoryTypeIdentifierSleepAnalysis,Apple Watch de Breno,10.2,NaN,2024-01-04 08:12:23 -0300,2024-01-04 07:55:31-03:00,2024-01-04 08:00:01-03:00,HKCategoryValueSleepAnalysisAwake,NaN,4.500000,0.075000,4.500000,minutes,2024-01-04
1948001,HKCategoryTypeIdentifierSleepAnalysis,Apple Watch de Breno,10.2,NaN,2024-01-04 08:12:23 -0300,2024-01-04 08:00:01-03:00,2024-01-04 08:08:01-03:00,HKCategoryValueSleepAnalysisAsleepCore,NaN,8.000000,0.133333,8.000000,minutes,2024-01-04
1948002,HKCategoryTypeIdentifierSleepAnalysis,Apple Watch de Breno,10.2,NaN,2024-01-04 08:12:23 -0300,2024-01-04 08:00:01-03:00,2024-01-04 08:08:01-03:00,HKCategoryValueSleepAnalysisInBed,NaN,8.000000,0.133333,8.000000,minutes,2024-01-04


In [107]:
df.loc[df['type'].isin(group_day_value_sum)].groupby(['type', 'sourceName', 'start_date', 'value_type', 'value'] )['value_fix'].sum()

type                                   sourceName            start_date  value_type  value                                        
HKCategoryTypeIdentifierSleepAnalysis  Apple Watch de Breno  2021-12-06  minutes     HKCategoryValueSleepAnalysisAsleepUnspecified    455.500000
                                                             2021-12-07  minutes     HKCategoryValueSleepAnalysisAsleepUnspecified    328.000000
                                                             2021-12-08  minutes     HKCategoryValueSleepAnalysisAsleepUnspecified    477.500000
                                                             2021-12-09  minutes     HKCategoryValueSleepAnalysisAsleepUnspecified    467.000000
                                                             2021-12-10  minutes     HKCategoryValueSleepAnalysisAsleepUnspecified    479.500000
                                                                                                                                         ...    